In [8]:
%pip install openai
%pip install llama-index

     |████████████████████████████████| 70 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 360 kB 3.9 MB/s eta 0:00:01
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     |████████████████████████████████| 61 kB 7.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 140 kB 3.2 MB/s eta 0:00:01
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
     |████████████████████████████████| 404 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 735 kB 4.7 MB/s eta 0:00:01
  Using cached typing_inspect-0.8.0-py3-none-any.whl (8.7 kB)
  Using cached marshmallow_enum-1.5.1-py2.py3-none-any.whl (4.2 kB)
  Using cached marshmallow-3.19.0-py3-none-any.whl (49 kB)
     |████████████████████████████████| 2.9 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 293 kB 5.4 MB/s eta 0:00:01
  Created wheel for l

In [1]:
import logging, sys

logging.disable(sys.maxsize)

In [2]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        'explaintext': True,
    }
).json()

page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [4]:
import os
os.environ['OPENAI_API_KEY'] =  os.environ['API_KEY'] 


In [9]:
from llama_index import GPTTreeIndex, SimpleDirectoryReader, LLMPredictor
# from llama_index.logger import LlamaLogger
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [10]:
# gpt-4
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))

In [11]:
documents = SimpleDirectoryReader('data').load_data()


In [12]:
index = GPTTreeIndex(documents, llm_predictor=llm_predictor_gpt4)



In [13]:
index.save_to_disk('index_gpt4.json')


In [14]:
new_index = GPTTreeIndex.load_from_disk('index_gpt4.json')


In [15]:
response_gpt4 = new_index.query(
    "What battles took place in New York City in the American Revolution?",
    llm_predictor=llm_predictor_gpt4,
    verbose=True
)

> Starting query: What battles took place in New York City in the American Revolution?
>[Level 0] Current response: ANSWER: 2

I selected summary 2 because it discusses New York City's history, including its role during the American Revolution. Although it does not specifically mention battles, it is the most relevant choice to the question as it provides context about the city's involvement in the war.
>[Level 0] Selected node: [2]/[2]
>[Level 0] Node [2] Summary text: New York City has a rich history, from its beginnings as a Dutch trading post to its growth into a global economic and cultural center. Slavery played a significant role in the city's early economy, and the African Burying Ground was discovered during construction in the 1990s. The American Revolution saw New York become a haven for Loyalist refugees and escaped slaves, and the city served as the national capital under the Articles of Confederation and the first capital under the U.S. Constitution.

Throughout the 19th 

In [16]:
str(response_gpt4)



'The context information mentions the Battle of Long Island as a battle that took place in New York City during the American Revolution.'